<h3> Części mowy (POS)

In [1]:
text = "The Very Large Telescope (VLT) of the European Southern Observatory (ESO), an array of four individual telescopes in the Atacama desert, has given us a huge amount of new data about the universe. Researchers have now used it to find a group of six galaxies around a supermassive black hole, from when the Universe was just 0.9 billion years old - it's estimated to be 13.8 billion years old now. Black holes are thought to sit at the center of galaxies including the Milky Way. "

In [2]:
from nltk import sent_tokenize, word_tokenize

In [3]:
sentences = sent_tokenize(text.lower()) #podzial na sentencje 
print(sentences)

['the very large telescope (vlt) of the european southern observatory (eso), an array of four individual telescopes in the atacama desert, has given us a huge amount of new data about the universe.', "researchers have now used it to find a group of six galaxies around a supermassive black hole, from when the universe was just 0.9 billion years old - it's estimated to be 13.8 billion years old now.", 'black holes are thought to sit at the center of galaxies including the milky way.']


In [4]:
tokens_sen1 = word_tokenize(sentences[0]) #podzial na slowa pierwszego zdania
print(tokens_sen1)

['the', 'very', 'large', 'telescope', '(', 'vlt', ')', 'of', 'the', 'european', 'southern', 'observatory', '(', 'eso', ')', ',', 'an', 'array', 'of', 'four', 'individual', 'telescopes', 'in', 'the', 'atacama', 'desert', ',', 'has', 'given', 'us', 'a', 'huge', 'amount', 'of', 'new', 'data', 'about', 'the', 'universe', '.']


In [5]:
from nltk.probability import FreqDist

fdist = FreqDist(tokens_sen1)
fdist

FreqDist({'the': 4, 'of': 3, '(': 2, ')': 2, ',': 2, 'very': 1, 'large': 1, 'telescope': 1, 'vlt': 1, 'european': 1, ...})

In [7]:
fdist.most_common(2) #dwa najczęstsze

[('the', 4), ('of', 3)]

In [9]:
from nltk import pos_tag #Części mowy

tags = pos_tag(tokens_sen1)
print(tags)

[('the', 'DT'), ('very', 'RB'), ('large', 'JJ'), ('telescope', 'NN'), ('(', '('), ('vlt', 'NN'), (')', ')'), ('of', 'IN'), ('the', 'DT'), ('european', 'JJ'), ('southern', 'JJ'), ('observatory', 'NN'), ('(', '('), ('eso', 'NN'), (')', ')'), (',', ','), ('an', 'DT'), ('array', 'NN'), ('of', 'IN'), ('four', 'CD'), ('individual', 'JJ'), ('telescopes', 'NNS'), ('in', 'IN'), ('the', 'DT'), ('atacama', 'NN'), ('desert', 'NN'), (',', ','), ('has', 'VBZ'), ('given', 'VBN'), ('us', 'PRP'), ('a', 'DT'), ('huge', 'JJ'), ('amount', 'NN'), ('of', 'IN'), ('new', 'JJ'), ('data', 'NNS'), ('about', 'IN'), ('the', 'DT'), ('universe', 'NN'), ('.', '.')]


<h4> Słowa nieistniejące 

In [10]:
tags2 = pos_tag(['gutvidual']) #a jednak przewiduje... :)
tags2

[('gutvidual', 'JJ')]

<h4> Słowa w zależności od kontekstu mogą mieć różne POS tagi

In [11]:
print(pos_tag(word_tokenize("I cannot bear this headache"))) #bear jest czasownikiem

[('I', 'PRP'), ('can', 'MD'), ('not', 'RB'), ('bear', 'VB'), ('this', 'DT'), ('headache', 'NN')]


In [12]:
print(pos_tag(word_tokenize("Yesterday, I saw a bear")))  #bear jest rzeczownikiem

[('Yesterday', 'NN'), (',', ','), ('I', 'PRP'), ('saw', 'VBD'), ('a', 'DT'), ('bear', 'NN')]


<h4> Uczenie HMM - jaki POS?

Problem:

- dana jest sekwencja wyrazów  $x_1x_2x_3...x_n$  (przykładowo ["I", "like", "dogs"])
- cel: znaleźć najbardziej prawdopodobną sekwencje tagów  $y_1y_2y_3...y_n$  (przykładowo ["PRP","VBP", "NNS"]), czyli:

$argmax_{y_1,y_2,y_3...,y_n}P(x_1,x_2,x_3,...,x_n,y_1,y_2,y_3,...,y_n)$

- Jak? 

Algorytm Viterbiego

- Stany:

obserwacje = słowa

stany ukryte = tagi POS


- Uczenie algorytmu polega na wyznaczeniu:

prawdopodobieństw przejść = prawdopodobieństwa typu P(VP|NP)

prawdopodobieńsw emisji = prawdopodobieństwa typu P(John|VP)

In [13]:
from nltk.corpus import treebank  #korpus do testowania algorytmow przewidujących częsci mowy

treebank.readme().replace('\n', ' ')[:260]

'[ PENN TREEBANK SAMPLE ]\r http://www.cis.upenn.edu/~treebank/home.html\r \r This is a ~5% fragment of Penn Treebank, (C) LDC 1995.  It is made\r available under fair use for the purposes of illustrating NLTK tools\r for tokenizing, tagging, chunking and parsing.  '

In [14]:
print(treebank.tagged_sents()[:2]) #pierwsze dwie otagowane sentencje

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')]]


In [15]:
len(treebank.tagged_sents()) #ile wszystkich otagowanych sentencji?

3914

<h4> HMM

In [16]:
from nltk.tag import hmm  #Ukryte modele Markowa

trainer = hmm.HiddenMarkovModelTrainer() #buduje model HMMM
tagger = trainer.train_supervised(treebank.tagged_sents()) #trenuje model (metoda największej wiarygodności) na wszystkich otagowanych sentencjach z treebank

In [18]:
tagger.tag(['today','is','such','a','very','nice','day']) #patrze jak model sprawdza sie na przykladowych danych

[('today', 'NN'),
 ('is', 'VBZ'),
 ('such', 'PDT'),
 ('a', 'DT'),
 ('very', 'RB'),
 ('nice', 'JJ'),
 ('day', 'NN')]

In [20]:
tagger.tag(['today','is','such','a','picturesque','day']) #patrze jak model sprawdza sie na innych przykladowych danych

[('today', 'NN'),
 ('is', 'VBZ'),
 ('such', 'PDT'),
 ('a', 'DT'),
 ('picturesque', 'NNP'),
 ('day', 'NNP')]

In [21]:
pos_tag(['today','is','such','a','picturesque','day']) #POS tagging przy uzyciu metody pos_tag, dla porownania

[('today', 'NN'),
 ('is', 'VBZ'),
 ('such', 'JJ'),
 ('a', 'DT'),
 ('picturesque', 'JJ'),
 ('day', 'NN')]

In [23]:
tagger.accuracy(treebank.tagged_sents())  #jaka jest przewidywalnosc modelu na całym korpusie treebank?

0.9815546902936152

<h4> Zadanie1: Podziel wyjściowy zbiór sentencji $treebank.tagged\_sents()$ na dwa zbiory: pierwsze 3000 sentencje zapisz pod zmienną  $trained$, natomiast pozostałe sentencje zapisz pod zmienną $tested$. Zbuduj model tylko w oparciu o $trained$. Następnie wykonaj ewaluacje modelu zarówno na  $trained$  jak i  $tested$. Zastanów się z czego mogą wynikać rozbieżności.

<h3> Porcjowanie (Chunking)


In [90]:
print(tags)

[('the', 'DT'), ('very', 'RB'), ('large', 'JJ'), ('telescope', 'NN'), ('(', '('), ('vlt', 'NN'), (')', ')'), ('of', 'IN'), ('the', 'DT'), ('european', 'JJ'), ('southern', 'JJ'), ('observatory', 'NN'), ('(', '('), ('eso', 'NN'), (')', ')'), (',', ','), ('an', 'DT'), ('array', 'NN'), ('of', 'IN'), ('four', 'CD'), ('individual', 'JJ'), ('telescopes', 'NNS'), ('in', 'IN'), ('the', 'DT'), ('atacama', 'NN'), ('desert', 'NN'), (',', ','), ('has', 'VBZ'), ('given', 'VBN'), ('us', 'PRP'), ('a', 'DT'), ('huge', 'JJ'), ('amount', 'NN'), ('of', 'IN'), ('new', 'JJ'), ('data', 'NNS'), ('about', 'IN'), ('the', 'DT'), ('universe', 'NN'), ('.', '.')]


In [91]:
from nltk.chunk import RegexpParser

grammar = "chunk: {<DT>?<JJ>*<NN>}" #DT - determiner/określnik, JJ - adjective/przymiotnik, NN - noun/rzeczonik
chunker = RegexpParser(grammar) 
result = chunker.parse(tags) #tags zdefiniowane bylo wyzej

In [92]:
print(result)

(S
  the/DT
  very/RB
  (chunk large/JJ telescope/NN)
  (/(
  (chunk vlt/NN)
  )/)
  of/IN
  (chunk the/DT european/JJ southern/JJ observatory/NN)
  (/(
  (chunk eso/NN)
  )/)
  ,/,
  (chunk an/DT array/NN)
  of/IN
  four/CD
  individual/JJ
  telescopes/NNS
  in/IN
  (chunk the/DT atacama/NN)
  (chunk desert/NN)
  ,/,
  has/VBZ
  given/VBN
  us/PRP
  (chunk a/DT huge/JJ amount/NN)
  of/IN
  new/JJ
  data/NNS
  about/IN
  (chunk the/DT universe/NN)
  ./.)


In [93]:
print(result[0]) #mozna sie odwolywac

('the', 'DT')


<h4> Zadanie2: Zapisz do listy wszystkie chunki występujące w tekście a spełniajacy zadany (przez siebie) warunek.

In [94]:
result.draw() #rysowanie tego co wyzej (pojawia sie nowe okno)

<h4> Zadanie3: Zinterpretuj poniższy chunk.

In [30]:
grammar = "chunk: {<NN.?>*<VBD.?>*<JJ.?>*<CC>?}" #co oznacza wyrazenie regularne zapisane w {}?
chunker = RegexpParser(grammar) 
result = chunker.parse(tags) #tags zdefiniowane bylo wyzej
result.draw()

<h4> Zadanie4: Pobierz dowolny tekst (przez f = open(...)). Następnie wyznacz dla niego zaproponowany przez siebie chunk.

<h3> Rozpoznawanie bytów (Entity Recognition)


In [95]:
tokens_sen2 = word_tokenize("Albert Einstein was born at Ulm, in Württemberg, Germany, on March 14, 1879.")
tags2 = pos_tag(tokens_sen2)
print(tags2)

[('Albert', 'NNP'), ('Einstein', 'NNP'), ('was', 'VBD'), ('born', 'VBN'), ('at', 'IN'), ('Ulm', 'NNP'), (',', ','), ('in', 'IN'), ('Württemberg', 'NNP'), (',', ','), ('Germany', 'NNP'), (',', ','), ('on', 'IN'), ('March', 'NNP'), ('14', 'CD'), (',', ','), ('1879', 'CD'), ('.', '.')]


In [96]:
from nltk import ne_chunk

namedEnt = ne_chunk(tags2, binary = True) #przeszukiwanie pod kątem osób, miejsc itd...
print(namedEnt)

(S
  (NE Albert/NNP Einstein/NNP)
  was/VBD
  born/VBN
  at/IN
  (NE Ulm/NNP)
  ,/,
  in/IN
  (NE Württemberg/NNP)
  ,/,
  (NE Germany/NNP)
  ,/,
  on/IN
  March/NNP
  14/CD
  ,/,
  1879/CD
  ./.)


In [97]:
namedEnt.draw()

<h4> Zadanie5: Znajdz wszystkie byty (entity) w tekście poniżej.

In [38]:
txt = "Google, LLC is an American multinational technology company that specializes in Internet-related services and products, which include online advertising technologies, a search engine, cloud computing, software, and hardware. It is considered one of the Big Five technology companies in the U.S. information technology industry, alongside Amazon, Facebook, Apple, and Microsoft."

<h3> Stemowanie (Stemming)

In [80]:
from nltk.stem import PorterStemmer

In [81]:
ps = PorterStemmer() #Tworze stemer

words = ["wait", "waiting", "waited", "waits"]

for elem in words:
    print(ps.stem(elem)) #stemming

wait
wait
wait
wait


In [87]:
#stemowanie zdan
print([ps.stem(elem) for elem in word_tokenize("The cell is the basic structural and functional unit of all forms of life.")])

['the', 'cell', 'is', 'the', 'basic', 'structur', 'and', 'function', 'unit', 'of', 'all', 'form', 'of', 'life', '.']


<h3> Lematyzacja (Lemmatization) - "lepszy Stemming"

In [83]:
from nltk.stem import WordNetLemmatizer

In [84]:
lemmatizer = WordNetLemmatizer() #tworze Lemmatyzator 
print(lemmatizer.lemmatize("cats")) #wywołuje metode lemmatize z tego Lemmatyzatora na słowie 'cat'

cat


In [43]:
print(lemmatizer.lemmatize("feet"))

foot


In [44]:
print(lemmatizer.lemmatize("better"))

better


In [45]:
print(lemmatizer.lemmatize("better", pos = 'r')) #n-noun/rzeczownik (domyslnie), a-adjective/przymiotnik, v-verb/czasownik, r-adverb/przyslowek

well


<h3> Sieć słów (Wordnet)

In [46]:
from nltk.corpus import wordnet as wn

In [47]:
wn.synsets("automobile") #jakie są synstety (zbiory synonimow) dla slowa automobile?

[Synset('car.n.01'), Synset('automobile.v.01')]

In [48]:
wn.synset('car.n.01').lemma_names() #jakie są słowa w synsecie car.n.01; to synset obejmujący różne synonimy (lemmas)

['car', 'auto', 'automobile', 'machine', 'motorcar']

In [49]:
wn.synset('car.n.01').definition() #jaka jest definicja tego synsetu?

'a motor vehicle with four wheels; usually propelled by an internal combustion engine'

In [50]:
wn.synset('car.n.01').examples() #przykladowe uzycie w zdaniu

['he needs a car to get to work']

In [51]:
wn.synsets("car") #jakie są synsety slowa car?

[Synset('car.n.01'),
 Synset('car.n.02'),
 Synset('car.n.03'),
 Synset('car.n.04'),
 Synset('cable_car.n.01')]

In [52]:
#jak wyglada te 5 synsetow?

for synset in wn.synsets("car"):
    print(synset.lemma_names())

['car', 'auto', 'automobile', 'machine', 'motorcar']
['car', 'railcar', 'railway_car', 'railroad_car']
['car', 'gondola']
['car', 'elevator_car']
['cable_car', 'car']


<h3> Hiponimy i hiperonimy

In [53]:
motorcar = wn.synset("car.n.01")

In [54]:
types_of_cars = motorcar.hyponyms() #hiponim to takie slowo ktorego znaczenie semanatyczne pochodzi od innego slowa (hiperonimu) np rozowy to kolor, gęś to ptak
print(types_of_cars[14]) #przykład hiponimu (synsetu zawierającego hiponimy)

Synset('jeep.n.01')


In [55]:
wn.synset('jeep.n.01').lemma_names() #slowa w synsecie jeep.n.01

['jeep', 'landrover']

In [56]:
#hiponimy dla slowa car (z synsetu car.n.01)
print([lemma.name() for synset in types_of_cars for lemma in synset.lemmas()])

['ambulance', 'beach_wagon', 'station_wagon', 'wagon', 'estate_car', 'beach_waggon', 'station_waggon', 'waggon', 'bus', 'jalopy', 'heap', 'cab', 'hack', 'taxi', 'taxicab', 'compact', 'compact_car', 'convertible', 'coupe', 'cruiser', 'police_cruiser', 'patrol_car', 'police_car', 'prowl_car', 'squad_car', 'electric', 'electric_automobile', 'electric_car', 'gas_guzzler', 'hardtop', 'hatchback', 'horseless_carriage', 'hot_rod', 'hot-rod', 'jeep', 'landrover', 'limousine', 'limo', 'loaner', 'minicar', 'minivan', 'Model_T', 'pace_car', 'racer', 'race_car', 'racing_car', 'roadster', 'runabout', 'two-seater', 'sedan', 'saloon', 'sport_utility', 'sport_utility_vehicle', 'S.U.V.', 'SUV', 'sports_car', 'sport_car', 'Stanley_Steamer', 'stock_car', 'subcompact', 'subcompact_car', 'touring_car', 'phaeton', 'tourer', 'used-car', 'secondhand_car']


In [57]:
vehicles = motorcar.hypernyms() #hiperonimy (synsety) dla car.n.01
print(vehicles)

[Synset('motor_vehicle.n.01')]


In [58]:
print(sorted([lemma.name() for synset in vehicles for lemma in synset.lemmas()])) #wszystkie hiperonimy

['automotive_vehicle', 'motor_vehicle']


<h4> Zadanie6: Znajdź wszystkie hiponimy i hiperonimy dla słowa $dog$. Zacznij od wyszukania synsetów dla tego słowa, wybierz pierwszy z nich.

<h3> Synonimy

In [59]:
synonyms = []
for syn in wn.synsets('girl'):
    for lemma in syn.lemmas(): 
        synonyms.append(lemma.name())

print(synonyms)

['girl', 'miss', 'missy', 'young_lady', 'young_woman', 'fille', 'female_child', 'girl', 'little_girl', 'daughter', 'girl', 'girlfriend', 'girl', 'lady_friend', 'girl']


<h4> Antonimy

In [60]:
antonyms = []
for syn in wn.synsets("girl"):
    for l in syn.lemmas():
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())

print(antonyms)

['male_child', 'boy', 'son', 'boy']


<h4> Zadanie7: Znajdź synonimy i antonimy słowa $happy$.

<h3> Metryka Wu-Palmera - podobieństwo synsetów


In [61]:
bike = wn.synsets('bicycle')[0]
boy = wn.synsets('boy')[0]
human = wn.synsets('human')[0]

print(bike)
print(boy)
print(human)

Synset('bicycle.n.01')
Synset('male_child.n.01')
Synset('homo.n.02')


In [62]:
wn.synset('homo.n.02').lemma_names() #zeby srpawdzic co tam jest

['homo', 'man', 'human_being', 'human']

In [63]:
bike.wup_similarity(human), boy.wup_similarity(human), boy.wup_similarity(bike) #licze na ile slowa (a formalnie synsety są podobne do siebie)

(0.34782608695652173, 0.5217391304347826, 0.4444444444444444)

<h4> Zadanie8: Porównaj podobieństwo słów dog, cat i fish.

<h3> Uzupełnienie: Gramatyki

https://coling.epfl.ch/TP/corr/TP-parsing-sol.php

In [64]:
import nltk
from nltk.grammar import CFG
from nltk.parse.generate import generate
grammar = nltk.CFG.fromstring("""
S -> A W | B S
W -> A S | B W |
A -> 'a'
B -> 'b'
""")

list(generate(grammar, depth = 4)) #depth = maksymalna glębokosc drzewa

[['a', 'b'], ['a'], ['b', 'a']]

In [65]:
list(generate(grammar, depth = 5))

[['a', 'a', 'a'],
 ['a', 'b', 'b'],
 ['a', 'b'],
 ['a'],
 ['b', 'a', 'b'],
 ['b', 'a'],
 ['b', 'b', 'a']]

<h4> Pytanie: Jak powstało "aaa"? Z kolei na liście wyżej brak "aba", dlaczego?

<h3> Uzupełnienie: przetwarzanie języka polskiego

https://spacy.io/models/pl

In [66]:
import spacy #biblioteka do przetwarzania języka naturalnego, cos jak nltk
nlp = spacy.load("pl_core_news_sm")

In [69]:
stopwords = nlp.Defaults.stop_words

In [68]:
stopwords

{'a',
 'aby',
 'ach',
 'acz',
 'aczkolwiek',
 'aj',
 'albo',
 'ale',
 'alez',
 'ależ',
 'ani',
 'az',
 'aż',
 'bardziej',
 'bardzo',
 'beda',
 'bede',
 'bedzie',
 'bez',
 'bo',
 'bowiem',
 'by',
 'byc',
 'byl',
 'byla',
 'byli',
 'bylo',
 'byly',
 'bym',
 'bynajmniej',
 'być',
 'był',
 'była',
 'było',
 'były',
 'będzie',
 'będą',
 'będę',
 'cala',
 'cali',
 'caly',
 'cała',
 'cały',
 'chce',
 'choć',
 'ci',
 'cie',
 'ciebie',
 'cię',
 'co',
 'cokolwiek',
 'coraz',
 'cos',
 'coś',
 'czasami',
 'czasem',
 'czemu',
 'czy',
 'czyli',
 'często',
 'daleko',
 'dla',
 'dlaczego',
 'dlatego',
 'do',
 'dobrze',
 'dokad',
 'dokąd',
 'dosc',
 'dość',
 'duzo',
 'dużo',
 'dwa',
 'dwaj',
 'dwie',
 'dwoje',
 'dzis',
 'dzisiaj',
 'dziś',
 'gdy',
 'gdyby',
 'gdyz',
 'gdyż',
 'gdzie',
 'gdziekolwiek',
 'gdzies',
 'gdzieś',
 'go',
 'godz',
 'i',
 'ich',
 'ile',
 'im',
 'inna',
 'inne',
 'inny',
 'innych',
 'iv',
 'ix',
 'iz',
 'iż',
 'ja',
 'jak',
 'jakas',
 'jakaś',
 'jakby',
 'jaki',
 'jakichs',
 'jaki

In [70]:
zdanie = nlp("Tam jest biały pies")
print(zdanie)

Tam jest biały pies


In [73]:
for token in zdanie:
    print(token.text, token.lemma_, token.pos_) 

Tam tam ADV
jest być AUX
biały biały ADJ
pies pies NOUN


In [74]:
[token.text for token in zdanie] #same tokeny 

['Tam', 'jest', 'biały', 'pies']

In [75]:
#tokenizacja ze względu na slowa i zdania 

doc = nlp("To jest samochód. Samochód jest niebieski.")

words = [token.text for token in doc]

sentences = [sent.text for sent in doc.sents]

# Wyświetlenie wyników
print("Tokeny (słowa):", words)
print("Tokeny (zdania):", sentences)

Tokeny (słowa): ['To', 'jest', 'samochód', '.', 'Samochód', 'jest', 'niebieski', '.']
Tokeny (zdania): ['To jest samochód.', 'Samochód jest niebieski.']


In [76]:
from spacy import displacy
displacy.render(zdanie, style="dep")

In [77]:
displacy.render(zdanie, style = 'dep', options = {"compact": True})

In [78]:
doc = nlp("Nazywam się Michał Żak. \
        Mam 28 lat i urodziłem się w Krakowie 7 września 2002.") 

for entity in doc.ents:
    print(entity.text, entity.label_)

Michał Żak persName
Krakowie placeName
7 września 2002 date


<h4> Pobieranie danych z internetu

In [11]:
import requests
from bs4 import BeautifulSoup

def tekstowo(tytul, autor):
    url = f"https://www.tekstowo.pl/piosenka,{autor},{tytul}.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    lyrics = soup.find("div", class_="song-text").text.strip()
    return lyrics

        
tekstowo("strange_days","the_doors")

"Tekst piosenki:\nStrange days have found us\nStrange days have tracked us down\nThey're goin' to destroy \nOur casual joys\nWe shall go on playing or find a new town\nYeah\n\nStrange eyes fill strange rooms\nVoices will signal their tired end\nThe hostess is grinning\nHer guests sleep from sinning\nHear me talk of sin and you know this is it\nYeah\n\nStrange days have found us\nAnd through their strange hours, we linger alone\nBodies confused\nMemories misused\nAs we run from the day to a strange night of stone\n\n\n\n\n\n\n\n\n\xa0\nDodaj adnotację do tego tekstu »\nHistoria edycji tekstu"

In [12]:
import requests
from bs4 import BeautifulSoup

def pobierz_recenzje_filmweb(film_id):
    url = f"https://www.filmweb.pl/film/{film_id}/reviews"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    recenzje = []
    for recenzja in soup.find_all('div', class_='flatReview__text'):
        recenzje.append(recenzja.text.strip())
        
    return recenzje

film_id = "Ojciec+chrzestny-1972-1089"  
recenzje = pobierz_recenzje_filmweb(film_id)
if recenzje:
    for idx, recenzja in enumerate(recenzje, start=1):
        print(f"Recenzja {idx}: {recenzja}\n")

Recenzja 1: "Wierzę w Amerykę". Tak zaczyna się jedno z najważniejszych dzieł kinematografii, film wielokrotnie nagradzany, dziś już kultowy i stanowiący podmiot analizy wielu kinomaniaków. Kopalnia cytatów,... więcej

Recenzja 2: "Ojca chrzestnego" Francisa Forda Coppoli widział lub chociaż słyszał o nim każdy szanujący się miłośnik kina. Minęło już niemal trzydzieści lat od premiery filmu twórcy "Czasu apokalipsy", a on... więcej

Recenzja 3: Film F. F. Coppoli "Ojciec chrzestny", zwycięzca wielu nagród, w tym Oscara za najlepszy film i scenariusz adaptowany, jest uznawany przez krytyków i opinię publiczną za jedno z największych arcydzieł... więcej

Recenzja 4: Ciężko jest mi pisać o prawdopodobnie najlepszym filmie w historii kina, jednak spróbuję. Oczywiście moje wypociny nie oddadzą prawdziwej jakości tego dzieła sztuki, ale mogą zachęcić do jego... więcej

Recenzja 5: Lata 70. XX wieku z pewnością były jednymi z najważniejszych okresów w historii kina. To w tym czasie powstały k

<h4> Zadanie9
    
- Z użyciem bibliotek spacy/nltk przeanalizuj teksty: Ogniem i mieczem oraz Tajemnicza wyspa. 
    
- Jakie top 10 słów niebędących stopwordsami występuje w obu lekturach najczęściej (podaj częstość, można zwizualizować)
    
- Jakie top 10 tagów występuje w obu lekturach najczęściej (podaj częstości)
    
   
    
Link do lektur:
    
https://clarin-pl.eu/dspace/handle/11321/110
    
https://clarin-pl.eu/dspace/handle/11321/465

<h4> Zadanie10: Wejdź na dowolną listę przebojów (np. https://www.rmfmaxx.pl/hopbec) a następnie pobierz teksty piosenek z kolejnych pozycji. 

- Zamień wszystkie wyrazy na rozpoczynające się z małej litery. 
- Dokonaj tokenizacji. 
- Wyznacz miarę Herdana dla każdego z tekstów.
- Narysuj zależność pomiędzy zajmowanym na liście miejscem a miarą Herdana. Wyznacz współczynnik korelacji. Czy występuje jakaś tendencja?